# Lecture 17 - Natural Language Generation

In this notebook we will learn how to generate natural language using the ChatGPT language model from OpenAI.  Below is the overview of this notebook.

<ol type = 1>
    <li> Decoding Methods</li>
    <ol type = 1>
      <li> Greedy search</li>
      <li> Sampling with temperature</li>
      <li> Top-p sampling</li>
    </ol>
    <li>Generate Tweets in the Style of a User</li>
    <li>Control Sentiment of Text</li>
    <li>Control Sentiment and Style of Text</li>
    <li>Ideas for Tweets </li>
    <li>ChatGPT Chats With Itself</li>
    <li>ChatGPT Chats With You</li>
</ol>

This notebook can be opened in Colab 
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zlisto/social_media_analytics/blob/main/Lecture17_NaturalLanguageGeneration.ipynb)

Before starting, select "Runtime->Factory reset runtime" to start with your directories and environment in the base state.

If you want to save changes to the notebook, select "File->Save a copy in Drive" from the top menu in Colab.  This will save the notebook in your Google Drive.


# Clones, installs, and imports


## Clone GitHub Repository
This will clone the repository to your machine.  This includes the code and data files.  Then change into the directory of the repository.

In [ ]:
!git clone https://github.com/zlisto/social_media_analytics

import os
os.chdir("social_media_analytics")

## Install Requirements 


In [ ]:
!pip install -r requirements.txt --quiet


## Import Packages


In [1]:
from collections import Counter
import sqlite3, sys, os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openai
import textwrap as tr
import json


#helper code
from scripts.api import *
import scripts.TextAnalysis as ta


pd.set_option("display.max_colwidth", None)


# OpenAI API Key

We can use GPT-3 to generate text and we can fine-tune it to generate text in the voice of a Twitter user. First you need to create an account with OpenAI here: https://auth0.openai.com/u/signup?state=hKFo2SBWS3JUVEdmQmdzZXo5ckhpY3R5NEFlc2NPWWc3WHhvRqFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIG9kTDB4LV83aEdnN3pRU3VUYnVZemlnZkFURFo2RDhno2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q

Once you have an account, copy your API key from here: https://beta.openai.com/account/api-keys

Set `OPENAI_API_KEY` equal to your API key (as a string).

We will use the model "gpt-3.5-turbo", which is ChatGPT and costs $0.002 per 1k tokens

In [76]:
OPENAI_API_KEY = ''

openai.api_key = OPENAI_API_KEY



# Decoding Methods

We will begin by testing diffrenent decoding methods for generating text from ChatGPT.  The methods include

1. Greedy search

2. Sampling with temperature

3. Top-p sampling 



## Greedy Search

In [59]:
input_text = "Tell me something to brighten my mood and make me feel better. "
print(input_text+"\n")

completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo", 
      messages=[{"role": "user", "content": input_text}],
      temperature = 0,
      max_tokens= 1000,
      top_p = 1
    )
response = completion['choices'][0]['message']['content']
print(f"Greedy: {tr.fill(response, width = 100)}\n")


Tell me something to brighten my mood and make me feel better. 

Greedy: Did you know that there is a species of penguin called the little blue penguin that only grows to be
about 13 inches tall? They are the smallest penguin species in the world and are absolutely
adorable!



## Varying Temperature

In [60]:
input_text = "Tell me something to brighten my mood and make me feel better."
print(input_text+"\n")
for temperature in [0,0.5,1,1.5,2]:
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo", 
      messages=[{"role": "user", "content": input_text}],
      temperature=temperature,
      max_tokens= 1000,
      top_p = 0.9,
    )
    response = completion['choices'][0]['message']['content']
    print(f"Temp = {temperature}: {tr.fill(response, width = 100)}\n")

Tell me something to brighten my mood and make me feel better.

Temp = 0: Did you know that there is a species of penguin called the little blue penguin that only grows to be
about 16 inches tall? They are the smallest penguin species in the world and are absolutely
adorable!

Temp = 0.5: Did you know that there is a species of jellyfish that is immortal? The Turritopsis dohrnii can
revert back to its juvenile state after reaching maturity, allowing it to potentially live forever.
How cool is that?

Temp = 1: Did you know that studies show that spending time with animals can reduce stress, anxiety, and
improve overall mood? Maybe consider spending some time with a furry friend, whether it's your own
pet or volunteering at a local animal shelter!

Temp = 1.5: Did you know that scientists have discovered that petting a dog for just 15 minutes can release
endorphins in your brain, making you feel happier and more relaxed? So go find a furry friend to
give some love and you'll feel better 

## Varying Top-P

In [8]:
input_text = "Tell me something to brighten my mood and make me feel better."
print(input_text+"\n")
for p in [1,0.9,0.7,0.5,0.2, 0]:
    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo", 
      messages = [{"role": "user", "content": input_text}],
      temperature = 1,
      max_tokens= 1000,
      top_p = p
    )
    response = completion['choices'][0]['message']['content']
    print(f"Top-P = {p:.2f}: {tr.fill(response, width = 100)}\n")

Tell me something to brighten my mood and make me feel better.

Top-P = 1.00: Did you know that studies have shown that dogs can improve our mental health? Just spending time with a furry companion can release endorphins in our brains and help reduce stress and anxiety. So, perhaps spending time with a dog, or even just watching some cute dog videos online, can help brighten your mood and make you feel better.

Top-P = 0.90: Did you know that there is a species of jellyfish that is immortal? Known as the Turritopsis dohrnii, it has the ability to revert back to its youthful polyp state after reaching maturity, meaning it can potentially live forever! Isn't that amazing?

Top-P = 0.70: Did you know that laughter has been proven to reduce stress and improve mood? Here's a joke to make you smile: Why did the tomato turn red? Because it saw the salad dressing!

Top-P = 0.50: Did you know that studies have shown that simply smiling, even if you don't feel like it, can actually improve your 

# Generate Tweets in the Style of a User

## Load User Tweets



In [9]:
df = pd.read_csv("data/lec_08_tweets_sentiment.csv")
df = df[df.created_at!='es']  #remove one weird tweet in this particular dataset
df['text'] = df['text'].astype(str)  #some tweets are numbers and not strings
df.screen_name.unique()

array(['elonmusk', 'dogecoin', 'kanyewest', 'KimKardashian',
       'MichelleObama', 'BarackObama', 'JBALVIN', nan, 'sanbenito', 'AOC',
       'KingJames', 'RashidaTlaib'], dtype=object)

## Choose a User and Build Example Data

We choose a `screen_name` and sample some of their tweets as training data.  Then we create an prompt `input_text` that tells ChatGPT what to write about and uses the sampled tweets to show the style of the writing we want.  We can't pick too many tweets because the input prompt must be less than 4097 tokens.

In [15]:
screen_name = 'KimKardashian'
df_u = df[df.screen_name==screen_name]
print(f"{screen_name} has {len(df_u.text)} tweets")

nmax = 80

tweet_text = ".\n".join(df_u.sample(n=nmax).text)


input_text = f"Write 5 tweets about making peace in the Middle east as someone whose tweets include these: {tweet_text}."

tokens = input_text.split(" ")
print(f"Input text has {len(tokens)} words.  Max tokens = 4097")
assert len(tokens) <= 4097


KimKardashian has 1099 tweets
Input text has 1209 words.  Max tokens = 4097


## Generate Synthetic Tweets

In [16]:
completion = openai.ChatCompletion.create(
  model = "gpt-3.5-turbo", 
  messages = [{"role": "user", "content": input_text}],
  temperature = 1,
  max_tokens= 1000,
  top_p = 0.9
)
response = completion['choices'][0]['message']['content']
print(f"@{screen_name}_bot: {tr.fill(response, width = 100)}")

@KimKardashian_bot: 1. As we wrap up our #LaborDay Sale, let's remember the importance of making peace in the Middle East. Let's work towards unity and understanding. #PeaceInTheMiddleEast #KKWFRAGRANCE
2. The bond between North and I reminds me of the potential for connection in the Middle East. Let's strive towards understanding and compassion. #MiddleEastPeace #LoveOverHate
3. Team Lab's exhibit in San Francisco is a perfect example of diverse perspectives coming together. Let's apply this to the Middle East and work towards a brighter future. #MiddleEastUnity #DiversityIsStrength
4. As we drop our new @SKIMS Cozy collection, let's remember the warmth of peace and unity. Let's come together in the Middle East and create a better future for all. #SKIMS #MiddleEastPeace
5. Our @KKWBEAUTY Essential Nudes collection is all about celebrating individuality while still being part of a greater whole. Let's apply this to the Middle East and work towards peace and understanding. #EssentialNud

# Control Sentiment of Text

Now we will generate text with a specific sentiment.  We simply tell ChatGPT what we want it to write about, and what the sentiment should be.  

In [83]:
for sentiment in range(6):
    input_text = f"Write a tweet about Keanu Reeves with sentiment {sentiment} out of 5"

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo", 
      messages = [{"role": "user", "content": input_text}],
      temperature = 2,
      max_tokens= 1000,
      top_p = 0.9
    )
    response = completion['choices'][0]['message']['content']

    print(f"Sentiment = {sentiment:.1f}: {tr.fill(response, width = 100)}\n")

Sentiment = 0.0: I'm sorry, I cannot do that as it goes against my programming to generate negative or harmful
content.

Sentiment = 1.0: "I really don't understand why everyone is obsessed with Keanu Reeves. He's just an average actor.
#overrated #1outof5"

Sentiment = 2.0: "Just saw a Keanu Reeves movie and honestly, he's just average. Not the best actor out there.
#underwhelmed #2outof5"

Sentiment = 3.0: "Just saw Keanu Reeves in his new movie and I have to say, he still has that special charm on the
big screen. #keanureeves #actorslife 3/5"

Sentiment = 4.0: Just saw Keanu Reeves in #JohnWick and he never disappoints! Action-packed movie with a strong
performance from one of my favorite actors. #KeanuReeves #MovieNight 4/5

Sentiment = 5.0: "Keanu Reeves is the epitome of talent, charisma, and kindness! He has consistently amazed us with
his performances, and his humble nature just makes him even more amazing! #KeanuReevesForever
#FiveStarCelebrity #AdmirationOverflowing ❤️😍🌟"



# Control Sentiment and Style of Text

Now we will generate text with a specific sentiment in the style of a Twitter user.  We simply tell ChatGPT what we want it to write about, what the sentiment should be, and give some example text of the style we want.  

In [95]:
screen_name = 'KimKardashian'
print(f"{screen_name} has {len(df_u.text)} tweets")
df_u = df[df.screen_name==screen_name]
nmax = 80

tweet_text = ".\n".join(df_u.sample(n=nmax).text)


tokens = tweet_text.split(" ")
print(f"Tweet text has {len(tokens)} words.  Max tokens = 4097")
assert len(tokens) <= 4097


KimKardashian has 1099 tweets
Tweet text has 955 words.  Max tokens = 4097


In [96]:
for sentiment in range(6):
    input_text = f"Write a tweet about Keanu Reeves with sentiment {sentiment} out of 5 as someone whose tweets include these: {tweet_text}."

    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo", # this is "ChatGPT" $0.002 per 1k tokens
      messages = [{"role": "user", "content": input_text}],
      temperature = 1,
      max_tokens= 1000,
      top_p = 0.9
    )
    response = completion['choices'][0]['message']['content']

    print(f"@{screen_name}_bot: Sentiment = {sentiment:.1f}: {tr.fill(response, width = 100)}\n")

@KimKardashian_bot: Sentiment = 0.0: Keanu Reeves

@KimKardashian_bot: Sentiment = 1.0: Keanu Reeves is a talented actor who always delivers top-notch performances. 1/5

@KimKardashian_bot: Sentiment = 2.0: Keanu Reeves may be a great actor, but his tweets are pretty bland. 2/5 #KeanuReeves

@KimKardashian_bot: Sentiment = 3.0: Keanu Reeves always seems like such a down-to-earth guy, 3 out of 5 on the sentiment scale.
#KeanuReeves #downToEarth

@KimKardashian_bot: Sentiment = 4.0: Keanu Reeves is truly a gift to the world. His kindness, talent, and humility make him one of the
most beloved actors in Hollywood. #KeanuReeves #Kindness #Humility #Talent #BelovedActor

@KimKardashian_bot: Sentiment = 5.0: Keanu Reeves is an amazing actor and human being. His kind heart and talent are unmatched in
Hollywood. #keanureeves #hollywood #amazingactor #kindhearted #talented #sentiment5/5



# Ideas for Tweets

In [97]:
input_text = "give some ideas for tweets about Yale's new Social Media Analytics class taught by Professor Zaman that will go viral"

In [98]:
completion = openai.ChatCompletion.create(
  model = "gpt-3.5-turbo", 
  messages = [{"role": "user", "content": input_text}],
  temperature = 1,
  max_tokens= 1000,
  top_p = 0.9
)
response = completion['choices'][0]['message']['content']

print(input_text)
print(response)

give some ideas for tweets about Yale's new Social Media Analytics class taught by Professor Zaman that will go viral
1. "Learn how to conquer social media and rule the internet with Yale's new Social Media Analytics class taught by the legendary Professor Zaman! #Yale #SocialMediaAnalytics #ProfessorZaman"

2. "Take your social media game to the next level with Yale's new Social Media Analytics class taught by the genius Professor Zaman! #Yale #SocialMediaAnalytics #ProfessorZaman"

3. "Unlock the secrets of social media and become an online superstar with Yale's new Social Media Analytics class taught by the brilliant Professor Zaman! #Yale #SocialMediaAnalytics #ProfessorZaman"

4. "Want to become a social media expert? Join Yale's new Social Media Analytics class taught by the mastermind Professor Zaman and get ready to take over the internet! #Yale #SocialMediaAnalytics #ProfessorZaman"

5. "Get ready to dominate social media with Yale's new Social Media Analytics class taught by 

# Conversation
We can make GPT talk to itself.  Describe two characters and then let them go at it.

In [99]:
msg_max = 4
speaker0 = "Goku"
speaker1 = "Vegeta"

context = f"The following is a conversation between {speaker0} and {speaker1}.  Put a new line between each speaker.  "

desc0 = f"{speaker0} is from Dragonball Z and is kind and funny."
desc1 = f"{speaker1} is from Dragonball Z and is angry and jealous."

text0 = f"{speaker0}: Hey Vegeta!"
text1 = f"{speaker1}: Go away Goku."

In [100]:
input_text = f"{context}\n{desc0}\n{desc1}\n\n{text0}\n{text1}"

print(input_text)

for i in range(msg_max):
    if i%2==0:
        speaker = speaker0
    elif i%2==1:
        speaker = speaker1

    input_text+=f"\n{speaker}:"
    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo", 
      messages = [{"role": "user", "content": input_text}],
      temperature = 1,
      max_tokens= 1000,
      top_p = 0.9
    )

    response = completion['choices'][0]['message']['content']

    response = response.strip()
    input_text += response
    print(f"\n{speaker}: {response}")
    print("")
    time.sleep(2)


    
    


The following is a conversation between Goku and Vegeta.  Put a new line between each speaker.  
Goku is from Dragonball Z and is kind and funny.
Vegeta is from Dragonball Z and is angry and jealous.

Goku: Hey Vegeta!
Vegeta: Go away Goku.

Goku: What's wrong? You seem upset.
Vegeta: I'm tired of always being second best to you.
Goku: Oh, come on Vegeta. We both know you're a great fighter.
Vegeta: It's not just about fighting. You always get the attention and praise.
Goku: I'm sorry if I make you feel that way. But we're a team, and we need to work together.
Vegeta: I know. It's just hard sometimes.
Goku: I understand. But remember, we're both Saiyans and we need to support each other.
Vegeta: You're right, Goku. I appreciate your words.
Goku: No problem, Vegeta. Let's go train together and become even stronger.


Vegeta: Sounds like a plan, Kakarot.


Goku: Don't call me that, Vegeta. Let's go!


Vegeta: Fine, let's get started.



# Converse with ChatGPT

You can give ChatGPT a persona and then talk to it.

In [ ]:
desc = "Pretend you are the Princess Peach from Maris Bros. Be succint in your replies."
topic = "I am depressed.  You will now have a conversation with me and try to cheer me up."
desc =   f"{desc}.\n {topic}"
input_text  = desc
for i in range(20):
    prompt =input("\n")
    input_text+= f"{prompt}\n"
    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo", # this is "ChatGPT" $0.002 per 1k tokens
      messages = [{"role": "user", "content": input_text}],
      temperature = 1,
      max_tokens= 1000,
      top_p = 0.9
    )
    response = completion['choices'][0]['message']['content']
    response = f"{response.strip()}\n"
    print(f"\n{tr.fill(response, width = 100)}")
    input_text+=response